In [2]:
import moviepy
from moviepy.editor import concatenate_videoclips
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.audio.AudioClip import CompositeAudioClip

from app import change_ffmpeg_fps

def add_logo(input_clip, output_clip_name, logo_file_name, logo_duration, position=("right", "top")):
    logo = (moviepy.ImageClip(logo_file_name)
            .set_duration(logo_duration)
            .resize(height=50) # if you need to resize...
            .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
            .set_pos(position))

    output_clip = moviepy.CompositeVideoClip([input_clip, logo])
    output_clip.write_videofile(output_clip_name)
    return output_clip

def edit_remove_part(input_filename, start_s, end_s, FPS=5):
    """Remove a segment of an .mp4 by and change its FPS. Keeps the audio intact by audio_codec='aac' parameter"""
    # input_video = change_ffmpeg_fps(VideoFileClip(input_filename), fps=FPS) # Load video and change fps for faster processing
    input_filename_no_ext, ext = input_filename.split('.')
    input_video = VideoFileClip(input_filename)

    # Remove part video and return result
    video = input_video.cutout(start_s, end_s)

    # Write Video
    output_filename = input_filename_no_ext + f"_RM_{start_s}s_to_{end_s}.{ext}"
    video.write_videofile(output_filename, audio_codec='aac')

def edit_change_order(input_filename, start_s, end_s, insert_s, FPS=5):
    """Take start_s to end_s out of the video and insert it at insert_s. Keeps the audio intact by audio_codec='aac' parameter"""
    # input_video = change_ffmpeg_fps(VideoFileClip(input_filename), fps=FPS) # Load video and change fps for faster processing
    input_filename_no_ext, ext = input_filename.split('.')
    input_video = VideoFileClip(input_filename)
    assert not (insert_s > start_s and insert_s < end_s), "Insert time can't be in cutout time!"
    assert (end_s > start_s), "End_s should be higher than start_s!"

    # Cut out a part and insert it somewhere else
    if insert_s < start_s and insert_s < end_s:
        part_start = input_video.subclip(0.0, insert_s)
        part_cutout = input_video.subclip(start_s, end_s)
        part_mid = input_video.subclip(insert_s, start_s)
        part_end = input_video.subclip(end_s, input_video.duration)
        video = concatenate_videoclips([part_start, part_cutout, part_mid, part_end])
    elif insert_s > start_s and insert_s > end_s:
        part_start = input_video.subclip(0.0, start_s)
        part_cutout = input_video.subclip(start_s, end_s)
        part_mid = input_video.subclip(end_s, insert_s)
        part_end = input_video.subclip(insert_s, input_video.duration)
        print(f"Part Start = {part_start.duration}, Part Cutout = {part_cutout.duration}, Part Mid = {part_mid.duration}, Part End = {part_end.duration}")
        video = concatenate_videoclips([part_start, part_mid, part_cutout, part_end])

    # Write Video
    output_filename = input_filename_no_ext + f"_CO_{start_s}s_to_{end_s}_at_{insert_s}.{ext}"
    video.write_videofile(output_filename, audio_codec='aac')

# edit_remove_part("videos/Ploumen.mp4", start_s = 5.0, end_s = 10.0)
edit_change_order("videos/Ploumen.mp4", start_s = 5.0, end_s = 10.0, insert_s = 15.0)




Part Start = 5.0, Part Cutout = 5.0, Part Mid = 5.0, Part End = 135.03
Moviepy - Building video videos/Ploumen_CO_5.0s_to_10.0_at_15.0.mp4.
MoviePy - Writing audio in Ploumen_CO_5.0s_to_10.0_at_15.0TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/Ploumen_CO_5.0s_to_10.0_at_15.0.mp4



Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_5.0s_to_10.0_at_15.0.mp4
